In [ ]:
# 크롬 드라이버 설치
import chromedriver_autoinstaller


# 셀레니움
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service


# bs4
from bs4 import BeautifulSoup


# ETC
from time import sleep
import os
from datetime import datetime
import openpyxl
from tqdm import tqdm

# 목차

1. chrome_driver_setting
2. excel_control
3. crawler

In [ ]:
# User Agent 자동으로 가져오기  << 해당 크롤러를 요청한 고객께서 python을 사용할줄 모르기 때문에 간단한 작업임에도 자동화를 시켰습니다.
def UA_crawler():
    service = Service(executable_path=chromedriver_autoinstaller.install(path = os.getcwd())) # 크롬 드라이버 설치

    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox') # 샌박끔        
    options.add_argument('--disable-dev-shm-usage') # /dev/shm 비활
    options.add_argument('headless') # 헤드리스
    options.add_argument('--blink-settings=imagesEnabled=false') # 이미지 출력 안함
    options.add_argument('--mute-audio') # 음소거
    options.add_argument('disable-gpu') # gpu 사용 해제



    browser = webdriver.Chrome(service=service, options=options)
    browser.get('https://www.whatismybrowser.com/detect/what-is-my-user-agent/') 
    browser.implicitly_wait(10)
    temp = BeautifulSoup(browser.page_source, 'html.parser')
    browser.close()
    UA = temp.find('div', attrs={'id' : 'detected_value'}).get_text()
    return UA

In [ ]:
# crawler 함수
def phonetic_alphabet(browser, word):
    browser.get(f'https://dic.daum.net/search.do?q={word}&dic=eng')
    browser.implicitly_wait(2)
    sleep(1)
    try:
        phonetic_alphabet = browser.find_element(By.CLASS_NAME, 'txt_pronounce').text
    except:
        phonetic_alphabet = f'dic.daum.net/search.do?q={word}&dic=eng'
    return phonetic_alphabet

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox') # 샌박끔        
options.add_argument('--disable-dev-shm-usage') # /dev/shm 비활
options.add_argument('headless') # 헤드리스
options.add_argument('--blink-settings=imagesEnabled=false') # 이미지 출력 안함
options.add_argument('--mute-audio') # 음소거
options.add_argument('disable-gpu') # gpu 사용 해제

UA = UA_crawler()
options.add_argument(f'--user-agent={UA}')
service = Service(executable_path=chromedriver_autoinstaller.install(path = os.getcwd())) # 크롬 드라이버 설치

try:
    browser = webdriver.Chrome(service=service, options=options)
    print("service")
except:
    browser = webdriver.Chrome(options=options)
    print("None")

In [ ]:
# Target Excel File
excel_file = 'voca.xlsx'
workbook = openpyxl.load_workbook(excel_file)

In [ ]:
######################################
############  크롤링 파트  ############
######################################

# 엑셀 내 시트 파일 마다 영어 단어가 적혀 있기에 모든 시트를 순회하며 영어 단어를 크롤링
for sheet in tqdm(workbook.sheetnames, desc="complete", total=len(workbook.sheetnames)):
    sh = workbook[sheet]
    # print("sheet이름: ",sheet) 
    for idx in range(2, sh.max_row + 1):
        if sh['E' + str(idx)].value is None and sh['E' + str(idx+1)].value is None: break

        sh['F' + str(idx)] = phonetic_alphabet(browser=browser, word=sh['E' + str(idx)].value)
    
    workbook.save("new_voca.xlsx") # 자동으로 저장

workbook.close()